In [1]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from datasets import load_dataset, list_datasets, load_metric, list_metrics
import torch
from tqdm import tqdm
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader

2022-04-10 14:02:00.516579: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
# Params
batch_size = 8


In [3]:
# Model and tokenizer
model_name = "google/pegasus-cnn_dailymail"
device = "cuda:3" if torch.cuda.is_available() else "cpu"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

In [4]:
# Load data
#xsum_train = load_dataset('xsum', split='train')
#xsum_valid = load_dataset('xsum', split='validation')
cnn_test = load_dataset('ccdv/cnn_dailymail','3.0.0', split='test')

Reusing dataset cnn_dailymail (/home/huangwenhao/.cache/huggingface/datasets/ccdv___cnn_dailymail/3.0.0/3.0.0/0107f7388b5c6fae455a5661bcd134fc22da53ea75852027040d8d1e997f101f)


In [5]:
cnn_test[0]

{'article': '(CNN)James Best, best known for his portrayal of bumbling sheriff Rosco P. Coltrane on TV\'s "The Dukes of Hazzard," died Monday after a brief illness. He was 88. Best died in hospice in Hickory, North Carolina, of complications from pneumonia, said Steve Latshaw, a longtime friend and Hollywood colleague. Although he\'d been a busy actor for decades in theater and in Hollywood, Best didn\'t become famous until 1979, when "The Dukes of Hazzard\'s" cornpone charms began beaming into millions of American homes almost every Friday night. For seven seasons, Best\'s Rosco P. Coltrane chased the moonshine-running Duke boys back and forth across the back roads of fictitious Hazzard County, Georgia, although his "hot pursuit" usually ended with him crashing his patrol car. Although Rosco was slow-witted and corrupt, Best gave him a childlike enthusiasm that got laughs and made him endearing. His character became known for his distinctive "kew-kew-kew" chuckle and for goofy catchph

In [6]:
# Data preprocess
#print(tokenizer(xsum_valid[0]['document'],truncation=True, padding="longest", return_tensors="pt"))
#valid_dataset = xsum_valid.map(lambda e: tokenizer(e['document'],truncation=True, padding="longest"), batch_size=batch_size, batched=True)
test_dataset = cnn_test.map(lambda e: tokenizer(e['article'],truncation=True, padding="longest"), batch_size=batch_size, batched=True)
#valid_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])


Loading cached processed dataset at /home/huangwenhao/.cache/huggingface/datasets/ccdv___cnn_dailymail/3.0.0/3.0.0/0107f7388b5c6fae455a5661bcd134fc22da53ea75852027040d8d1e997f101f/cache-089151cda8e74f79.arrow


In [7]:
#valid_ld = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_ld = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [8]:
max_len = 0
for batch in test_ld:
    max_len = max(max_len,batch['input_ids'].shape[1])

In [9]:
max_len

1024

In [10]:
test_predict = []
for batch in tqdm(test_ld):
    #print(batch['input_ids'].shape)
    batch = {x:y.to(device) for x,y in batch.items()}
    #print(batch)
    translated = model.generate(**batch)
    test_predict.extend(tokenizer.batch_decode(translated, skip_special_tokens=True, max_len=128))

100%|██████████| 1437/1437 [1:42:59<00:00,  4.30s/it]


In [11]:
max_len = 0
for s in test_predict:
    max_len = max(max_len, len(tokenizer.tokenize(s)))
print(max_len)

125


In [15]:
max_len = 0
for s in cnn_test['article']:
    max_len = max(max_len, len(tokenizer.tokenize(s)))
print(max_len)


2931


In [22]:
#Metric
Rouge = load_metric('rouge')
result = Rouge.compute(predictions=test_predict, references=[tokenizer.convert_tokens_to_string(tokenizer.tokenize(s)[:125]) for s in cnn_test['highlights']])
print(result)

{'rouge1': AggregateScore(low=Score(precision=0.39798612545719997, recall=0.4652249024453509, fmeasure=0.4167414644896658), mid=Score(precision=0.40047348757893625, recall=0.4679537979883459, fmeasure=0.4189110684348778), high=Score(precision=0.4030078241252382, recall=0.4705172512972913, fmeasure=0.4210992316564528)), 'rouge2': AggregateScore(low=Score(precision=0.19206276184131177, recall=0.22278394576855678, fmeasure=0.20023631100621667), mid=Score(precision=0.19447389175080337, recall=0.22571979566088857, fmeasure=0.20270136553064427), high=Score(precision=0.1969671639045246, recall=0.22838000744427847, fmeasure=0.20530494049373088)), 'rougeL': AggregateScore(low=Score(precision=0.2829065466577277, recall=0.33129641351980643, fmeasure=0.296421694667873), mid=Score(precision=0.2854077222476798, recall=0.3337014115549679, fmeasure=0.2986296996461908), high=Score(precision=0.28793935118177966, recall=0.33644750605785556, fmeasure=0.30104963867769563)), 'rougeLsum': AggregateScore(low=

In [20]:
tokenizer.convert_tokens_to_string(tokenizer.tokenize(cnn_test['highlights'][0]))

'James Best, who played the sheriff on "The Dukes of Hazzard," died Monday at 88 . "Hazzard" ran from 1979 to 1985 and was among the most popular shows on TV .'

In [ ]:
with open('predict_0.txt','w',encoding='utf-8') as f:
    for i in test_predict:
        f.write(i+'\n')